In [ ]:
import requests
from bs4 import BeautifulSoup 
import random
import pandas as pd
import json
import os
import urllib.request 
from time import sleep
import subprocess

In [ ]:
# Constants
# Export folder
DATA_FOLDER = "eap_data"
if not os.path.exists(DATA_FOLDER):
    os.makedirs(DATA_FOLDER)
DOWNLOAD_REPORT_FILE = DATA_FOLDER + "/download_report.csv"

In [ ]:
def get_page(url):
    try:
        user_agent_list = ['Mozilla/5.0 (compatible; MSIE 9.0; AOL 9.7; AOLBuild 4343.19; Windows NT 6.1; WOW64; Trident/5.0; FunWebProducts)',
                   'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',
                   'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/532.2 (KHTML, like Gecko) ChromePlus/4.0.222.3 Chrome/4.0.222.3 Safari/532.2',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; AS; rv:11.0) like Gecko',
                   'Mozilla/5.0 (Windows; U; Windows NT 6.1; rv:2.2) Gecko/20110201',
                   'Mozilla/5.0 (X11; Linux x86_64; rv:17.0) Gecko/20121202 Firefox/17.0 Iceweasel/17.0.1',
                   'Opera/9.80 (X11; Linux i686; Ubuntu/14.10) Presto/2.12.388 Version/12.16',
                   'Mozilla/5.0 (Windows; U; Windows NT 6.2; WOW64; rv:1.8.0.7) Gecko/20110321 MultiZilla/4.33.2.6a SeaMonkey/8.6.55',
                   'Mozilla/5.0 (Linux; U; Android 4.0.3; ko-kr; LG-L160L Build/IML74K) AppleWebkit/534.30 (KHTML, like Gecko) Version/4.0 Mobile Safari/534.30',
                   'Opera/12.02 (Android 4.1; Linux; Opera Mobi/ADR-1111101157; U; en-US) Presto/2.9.201 Version/12.02',
                   'Mozilla/5.0 (Macintosh; U; PPC Mac OS X Mach-O; XH; rv:8.578.498) fr, Gecko/20121021 Camino/8.723+ (Firefox compatible)',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36',
                   'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246',
                   'Mozilla/5.0 (X11; U; Linux x86_64; it-it) AppleWebKit/534.26+ (KHTML, like Gecko) Ubuntu/11.04 Epiphany/2.30.6',
                   'Mozilla/5.0 (X11; U; Linux x86_64; fr-FR) AppleWebKit/534.7 (KHTML, like Gecko) Epiphany/2.30.6 Safari/534.7',
                   'Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/534.7 (KHTML, like Gecko) Epiphany/2.30.6 Safari/534.7',
                   'Mozilla/5.0 (X11; U; Linux i686; sv-se) AppleWebKit/531.2+ (KHTML, like Gecko) Safari/531.2+ Epiphany/2.30.6',
                   'Mozilla/5.0 (Windows NT 6.3; rv:36.0) Gecko/20100101 Firefox/36.0',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10; rv:33.0) Gecko/20100101 Firefox/33.0',
                   'Mozilla/5.0 (X11; Linux i586; rv:31.0) Gecko/20100101 Firefox/31.0',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20130401 Firefox/31.0',
                   'Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0',
                   'Mozilla/5.0 (X11; U; Linux x86_64; en-US; rv:1.9.1.13) Gecko/20100916 Iceape/2.0.8',
                   'Mozilla/5.0 (X11; Linux x86_64; rv:17.0) Gecko/20121201 icecat/17.0.1',
                   'Mozilla/5.0 (X11; Linux i686; rv:7.0.1) Gecko/20111106 IceCat/7.0.1',
                   'Mozilla/5.0 (X11; Linux i686; rv:15.0) Gecko/20100101 Firefox/15.0.1 Iceweasel/15.0.1',
                   'Mozilla/5.0 (compatible, MSIE 11, Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko',
                   'Mozilla/5.0 (compatible; MSIE 10.6; Windows NT 6.1; Trident/5.0; InfoPath.2; SLCC1; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729; .NET CLR 2.0.50727) 3gpp-gba UNTRUSTED/1.0',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.4 (KHTML, like Gecko) Chrome/22.0.1250.0 Iron/22.0.2150.0 Safari/537.4'
                   ]
        random_user_agent = random.choice(user_agent_list)
        headers = {
            'User-Agent': random_user_agent,
        }
        page = requests.get(url, headers=headers)        
        if page.status_code != 200:
            raise Exception("Unable to get page content") 
    except:
      raise Exception("Unable to get page content") 
    return page.content
    

In [ ]:
def get_works_on_a_page(html_page):
    works = html_page.find_all("li", {"class": "pnl"})
    works_on_a_page = []
    for work in works:
        title = work.find_all("h3", {"class": "pnl-title"})        
        work_link = work.find("a")
        work_url = work_link['href']
        work_identifier = work_link['href']        
        work_identifier = work_identifier.replace("https://eap.bl.uk/archive-file/", "").strip()
        work_dict = {"Identifier": work_identifier, "location_url": work_url}
        works_on_a_page.append(work_dict)
    return works_on_a_page

In [ ]:
def get_work_metadata(work_dict, page_html):
    html_page = BeautifulSoup(page_html, "html.parser")
    page_title = html_page.find("h1", {"class": "page-title"})
    if page_title != None:
        work_dict["Title"] = page_title.string
    
    work_metadata_block = html_page.find("div", {"class": "block-archive-sidebar-block"})
    fields = work_metadata_block.find_all("div", {"class": "field"})
    for field in fields:
        field_label_ele = field.find("div", {"class": "field__label"})
        if field_label_ele != None:
            field_label = field_label_ele.string
            field_value_ele = field_label_ele.findNext('div')
            field_value = ""
            if field_value_ele != "None":
                field_value = field_value_ele.string
            field_label = field_label.replace(":", "")
            work_dict[field_label] = field_value

    work_hierarchy_block = html_page.find("div", {"class": "block-archive-hierarchy-block"})
    hierarchy_items = work_hierarchy_block.find_all("div", {"class": "hierarchy-item"})

    project_links = []
    project_link_labels = []
    collection_links = []
    collection_links_labels = []
    for hierarchy_item in hierarchy_items:
        related_item_link_ele = hierarchy_item.find("a")
        if related_item_link_ele != None:
            related_item_link = related_item_link_ele["href"]
            related_item_link_text = related_item_link_ele.contents[0]
            if "project" in related_item_link:
                project_links.append(related_item_link)
                project_link_labels.append(related_item_link_text)
            if "collection" in related_item_link:
                collection_links.append(related_item_link)
                collection_links_labels.append(related_item_link_text)
    
    work_dict["related_projects"] = project_links
    work_dict["related_projects_labels"] = project_link_labels
    work_dict["related_collections"] = collection_links
    work_dict["related_collections_labels"] = collection_links_labels
    return work_dict

In [ ]:
def download_work_files(work_dict):
    Identifier = work_dict["Identifier"]
    location_url = work_dict["location_url"]
    work_manifest_url = location_url + "/manifest"
    manifest_page = get_page(work_manifest_url)
    work_manifest_json = json.loads(manifest_page)

    # Add additional metadata from manifest
    metadata = work_manifest_json["metadata"]
    for metadata_item in metadata:
        if not metadata_item['label'] in work_dict.keys():
            work_dict[metadata_item['label']] = metadata_item['value']

    # Create folder to download work files to 
    file_name_prefix = Identifier.replace("/", "_")
    work_folder = DATA_FOLDER + "/" + file_name_prefix
    if not os.path.exists(work_folder):
        os.makedirs(work_folder)

    canvases = work_manifest_json["sequences"][0]["canvases"]

    success_flag = True
    for index, canvas_item in enumerate(canvases):
        image = canvas_item["images"][0]["resource"]
        image_id = canvas_item["images"][0]["resource"]["@id"]

        target_image = image_id.replace("full/full/0/default.jpg", "info.json")

        Identifier = work_dict["Identifier"]
        file_name = file_name_prefix + "_" + str(index + 1) + ".jpg"
        download_location = work_folder + "/" + file_name
        dezoomify_cmd = "/home/nat/tools/dezoomify/dezoomify-rs --largest --retries 3 --min-interval 100ms  --connect-timeout 10s --retry-delay 2s --timeout 60s " + target_image +  " " + file_name
        list_cmd = dezoomify_cmd.split()
        p1 = subprocess.run(list_cmd, capture_output=True, text=True)
        if "successfully saved to " in p1.stdout or "image was still created" in p1.stdout:
             p2 = subprocess.run(["mv", file_name, download_location] , capture_output=True, text=True)
        else:
            success_flag = False
            print("Error in downloading file " + target_image)
            print(p1.stdout)

    work_dict["Download_Status"] = success_flag
        
    return work_dict

In [ ]:
# Load download status file
if not os.path.exists(DOWNLOAD_REPORT_FILE):
    work_metadata_df = pd.DataFrame(columns=['Identifier','location_url'])
else:
    work_metadata_df = pd.read_csv(DOWNLOAD_REPORT_FILE)
work_metadata_df.shape

In [ ]:
for page_number in list(range(15, 20)): 
    url = "https://eap.bl.uk/search?f%5B0%5D=languages%3ATamil&f%5B1%5D=type%3AFile&page=" + str(page_number)
    page = get_page(url)
    html_page = BeautifulSoup(page, "html.parser")
    works_on_a_page = get_works_on_a_page(html_page)
    for work_dict in works_on_a_page:
        work_identifier = work_dict["Identifier"]
        # If we have not downloaded this work, then 
        if work_identifier not in work_metadata_df["Identifier"].values:
            print("Processing work with id: " + work_identifier)
            # Go to the work file and get the metadata
            work_url = work_dict["location_url"]
            try:
                page_html = get_page(work_url)
                work_dict = get_work_metadata(work_dict, page_html)
                work_dict = download_work_files(work_dict)
            except:
                print("ERROR - Unable to process " + work_identifier)  
                work_dict["Download_Status"] = False
                
            work_metadata_df = pd.concat([work_metadata_df, pd.DataFrame([work_dict])], ignore_index=True)
            work_metadata_df.to_csv(DOWNLOAD_REPORT_FILE, index=False)
    print("Completed processing page " + str(page_number))

In [ ]:
work_metadata_df.to_csv(DOWNLOAD_REPORT_FILE, index=False)